# tidyX examples

In [1]:
# pip install tidyX==1.6.3

In [2]:
!pip show tidyX

Name: tidyX
Version: 1.6.3
Summary: Python package to clean raw tweets for ML applications
Home-page: 
Author: Lucas Gómez Tobón, Jose Fernando Barrera
Author-email: lucasgomeztobon@gmail.com, jf.barrera10@uniandes.edu.co
License: MIT
Location: c:\users\lucas\anaconda3\envs\bx\lib\site-packages
Requires: emoji, nltk, numpy, pandas, regex, spacy, thefuzz, Unidecode
Required-by: 


In [3]:
from tidyX import TextPreprocessor as tp
from tidyX import TextNormalization as tn
from tidyX import TextVisualizer as tv

## Stemming and Lemmatizing Texts Efficiently

## Tutorial: Word Cloud

In [ ]:
import os
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import spacy
 
os.getcwd()

In [ ]:
tweets = pd.read_excel(r"../../../data/Tweets sobre venezuela.xlsx")
tweets.head()

In [ ]:
# Combine all documents into a single string
text = " ".join(doc for doc in tweets['Snippet'])

# Generate a word cloud image
wordcloud = WordCloud(background_color = "white", width = 800, height = 400).generate(text)

# Display the generated image
plt.figure(figsize=(10, 5))
plt.title("WordCloud before tidyX")
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis("off");

In [ ]:
tweets['clean'] = tweets['Snippet'].apply(lambda x: tp.preprocess(x, delete_emojis = False, extract = False,
                                                                  remove_stopwords = True))
tweets

In [ ]:
token_df = tp.unnest_tokens(df = tweets.copy(), input_column = "clean", id_col = None, unique = True)
token_df

In [ ]:
# Load spacy's model
model = spacy.load('es_core_news_lg')

In [ ]:
# Apply spanish_lemmatizer function to lemmatize the token
token_df["lemma"] = token_df["clean"].apply(lambda x: tn.lemmatizer(token = x, model = model))
token_df

In [ ]:
token_df["lemma"] = token_df["lemma"].apply(lambda x: tp.remove_words(x, remove_stopwords = True))
token_df = token_df[["clean", "lemma"]]
token_df

In [ ]:
tweets_long = tp.unnest_tokens(df = tweets.copy(), input_column = "clean", id_col = None, unique = False)
tweets_long 

In [ ]:
tweets_clean2 = tweets_long.merge(token_df, how = "left", on = "clean").groupby(["Snippet", "id"])["lemma"].agg(lambda x: " ".join(x)).reset_index()
tweets_clean2

In [ ]:
tweets_clean2['lemma'] = tweets_clean2['lemma'].apply(lambda x: tp.remove_extra_spaces(x))

In [ ]:
# Combine all documents into a single string
text = " ".join(doc for doc in tweets_clean2['lemma'])

# Generate a word cloud image
wordcloud = WordCloud(background_color = "white", width = 800, height = 400).generate(text)

# Display the generated image
plt.figure(figsize=(10, 5))
plt.title("WordCloud after tidyX")
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis("off");